In [1]:
#!/usr/bin/python2.7
import sys
sys.path.insert(0, '/home/torobo/catkin_ws/src/torobo_robot/torobo_rnn/scripts')
sys.path.insert(0, '/home/torobo/catkin_ws/src/tutorial/PVRNN/scripts')

from enum import Enum
import random
from torobo_driver import torobo_easy_command
from termcolor import colored
import pickle

import os
import numpy as np
import rospy
import math
import matplotlib.pyplot as plt
from sensor_msgs.msg import Image
import ctypes
from utils2 import Utils
import utils2
from NRL import NRL
from collections import deque
import time
from torobo_operator import ToroboOperator
from torobo_rnn_utils__upd3 import *


def to_radians(input_data):
    data=input_data
    for i in range(data.shape[0]):
    #pos_list=data[i]
        for j in range(len(data[i])):
            data[i][j]=math.radians(data[i][j])
    return data

def model_to_radians(input_data):
    for i in range(len(input_data)):
        input_data[i]=math.radians(input_data[i])
    return input_data


class ExtForce(object):
    # SETS THE KINEMATIC MODEL PARAMETERS (BOTH IN HENDRY'S AND FACTORY FIRMWARE)

    class Mode(Enum):
        Teaching = 0
        Experiment = 1

    def runCommands(self, _commandList, _controller):
          for command in _commandList:
                torobo_easy_command.SendEasyCommandText(_controller, command)
                rospy.sleep(0.01)

    def registerParameters(self, tau_th, kp, sum_e_max, d, kr, theta_th):
            # left and right arm 
            for i in range(0,6):
                
                # constructing the commands for the left arm
                commandList = []				
                commandList.append("param " + str(i+1) + " fftauth " +       str(tau_th[i]))
                commandList.append("param " + str(i+1) + " ffkp " +          str(kp[i]))
                commandList.append("param " + str(i+1) + " ffsigmaemax " +   str(sum_e_max[i]))
                commandList.append("param " + str(i+1) + " ffdamping " +     str(d[i]))
                commandList.append("param " + str(i+1) + " ffkr " +          str(kr[i]))
                commandList.append("param " + str(i+1) + " softki " +        str(theta_th[i]))	


                # send to execution
                self.runCommands(commandList, "left_arm_controller")

            # right arm 
            for i in range(0,6):
                ii = i + 6 
                # constructing the commands for the right arm
                commandList = []
                commandList.append("param " + str(i+1) + " fftauth " +       str(tau_th[ii]))
                commandList.append("param " + str(i+1) + " ffkp " +          str(kp[ii]))
                commandList.append("param " + str(i+1) + " ffsigmaemax " +   str(sum_e_max[ii]))
                commandList.append("param " + str(i+1) + " ffdamping " +     str(d[ii]))
                commandList.append("param " + str(i+1) + " ffkr " +          str(kr[ii]))
                commandList.append("param " + str(i+1) + " softki " +        str(theta_th[ii]))	

                # send to execution
                self.runCommands(commandList, "right_arm_controller")
                            
            for i in range(0,4):
                ii = i + 12 
                # constructing the commands for the head-torso chain
                commandList = []				
                commandList.append("param " + str(i+1) + " fftauth " +       str(tau_th[ii]))
                commandList.append("param " + str(i+1) + " ffkp " +          str(kp[ii]))
                commandList.append("param " + str(i+1) + " ffsigmaemax " +   str(sum_e_max[ii]))
                commandList.append("param " + str(i+1) + " ffdamping " +     str(d[ii]))
                commandList.append("param " + str(i+1) + " ffkr " +          str(kr[ii]))
                commandList.append("param " + str(i+1) + " softki " +        str(theta_th[ii]))	

                # send to execution
                self.runCommands(commandList, "torso_head_controller")


    def __init__(self, _mode):	
        
        if _mode == ExtForce.Mode.Teaching:

            tau_th = [2.0, 2.0, 1.0, 1.0, 0.5, 0.5,            2.0, 2.0, 1.0, 1.0, 0.5, 0.5,      20.5, 20.5, 20.5, 20.5]
            kp = [0.1, 0.1, 0.05, 0.05, 0.1, 0.1,            0.1, 0.1, 0.05, 0.05, 0.1, 0.1,        0.0,0.0,0.0,0.0]
            sum_e_max = [200.0,200.0,100.0,100.0,50.0,50.0,  200.0,200.0,100.0,100.0,50.0,50.0,   200.0,200.0,50.0,50.0]
            d = [0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9]
            # kr = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
            kr = [0.1]*16
            kr[5] = 0.3
            kr[11] = 0.3
            theta_th = [1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57]
            theta_th = [i*1.63 for i in theta_th]

            self.registerParameters( tau_th, kp, sum_e_max, d, kr, theta_th)


        elif _mode == ExtForce.Mode.Experiment:

            tau_th = [12.0,12.0,5.0,5.0,1.5,1.5,12.0,12.0,5.0,5.0,1.5,1.5,200.0,200.0,200.0,200.0]
            kp = [0.025,0.025,0.05,0.05,0.1,0.1,0.025,0.025,0.05,0.05,0.1,0.1,0.025,0.025,0.05,0.05]
            sum_e_max = [200.0,200.0,100.0,100.0,50.0,50.0,200.0,200.0,100.0,100.0,50.0,50.0,200.0,200.0,50.0,50.0]
            d = [0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9]
            kr = [0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
            # kr = [0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9]
            theta_th = [1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57,1.57]
            theta_th = [i*1.63 for i in theta_th]

            self.registerParameters( tau_th, kp, sum_e_max, d, kr, theta_th)
            

class Trial(object):
    def __init__(self, prop_path):
        self.nrl = NRL()
        self.cwd = os.getcwd()
        self.ut = Utils(self.cwd)
        self.nZ = '4,1'
        self.nD = '40,10'             
        #nZ = '4,2,1'
        #nD = '40,20,10'
        self.nrl.newModel(prop_path.encode('ascii'))
        self.nrl.load()
        self.nDof = self.nrl.getNDof()
        self.stateBufferSize = self.nrl.getStateBufferSize(self.nD, self.nZ, self.ut.delimiter)
        if self.nDof > 0:
            winSize = 15
            winBufferSize = winSize * self.nDof
            winBuffdata = deque(maxlen=winSize) # circular buffer
            primId = 2

            # The e_w parameters set bellow assume the network has two layers
            # as in the original distribution of the sources
            # in case more layers are set by changing the properties.d file,
            # the same dimension for e_w must be considered
            e_w = [0.025,0.025]
            self.start=False

            expTimeSteps = 30
            postdiction_epochs = 15
            alpha = 0.1
            beta1 = 0.9
            beta2 = 0.999
            storeStates = False
            storeER = False
            showERLog = False
            self.nrl.e_enable(primId,\
                         winSize,
                         (ctypes.c_float * len(e_w))(*e_w),
                         expTimeSteps,
                         postdiction_epochs,
                         (ctypes.c_float)(alpha),
                         (ctypes.c_float)(beta1),
                         (ctypes.c_float)(beta2), 
                         storeStates, 
                         storeER)

    def generate_pos(self):
        self.tgt_pos_buffer = np.zeros((self.nDof,), dtype=float)
        dataOut = (ctypes.c_float * self.nDof)(*self.tgt_pos_buffer)
        self.stateBufferSize = self.nrl.getStateBufferSize(self.nD, self.nZ, self.ut.delimiter)
        self.nrl.e_generate(dataOut)
        self.tgt_pos = np.frombuffer(dataOut, np.float32)
        return self.tgt_pos

    
    
def gostart(torobo):
    """ GO TO STARTING POSITION """
    idx = 4
    TIME = 5
    try:
        with HiddenPrints():
            servo_on(torobo)
            set_softness_override(torobo, 10.0)
            set_velocity_override(torobo, 10.0)
            with open('rad.pickle', 'rb') as f:
                RAD = pickle.load(f)

            left_arm_positions = RAD[idx][0:6]
            right_arm_positions = RAD[idx][6:12]
            torso_head_positions = RAD[idx][12:16]

            torobo.move(ToroboOperator.LEFT_ARM, positions=left_arm_positions, duration=TIME)
            torobo.move(ToroboOperator.RIGHT_ARM, positions=left_arm_positions, duration=TIME)
            torobo.move(ToroboOperator.TORSO_HEAD, positions=torso_head_positions, duration=TIME)
            rospy.sleep(TIME+0.1)
        print(colored('SUCCESS', 'white', 'on_green'))
    except:
        print('ERROR')
    

def setModeOverrides(torobo):
    """SET THE NEEDED KINEMATIC PARAMETERS AND MODES, OVERRIDES ON THE RIGHT JOINTS"""
    print('SETTING MODE: set_external_force_following_online_trajectory_control')
    with HiddenPrints():
        set_external_force_following_online_trajectory_control(torobo)
        torobo.set_control_mode(ToroboOperator.TORSO_HEAD, 'all', 'position')
    print('SETTING DYNAMIC PARAMETERS')
    with HiddenPrints():
        time.sleep(1)
        ExtForce(ExtForce.Mode.Teaching)
        time.sleep(1)
    print('OVERRIDES SET')
    with HiddenPrints():
        set_softness_override(torobo, 10.0)
        set_velocity_override(torobo, 10.0)
        
torobo = ToroboOperator()

In [ ]:
with HiddenPrints():
    torobo.set_control_mode(ToroboOperator.TORSO_HEAD, 'all', 'position')
    torobo.set_control_mode(ToroboOperator.LEFT_ARM, 'all', 'position')
    torobo.set_control_mode(ToroboOperator.RIGHT_ARM, 'all', 'position')

    time.sleep(1)
    move_startpos(torobo)

In [ ]:
# PLAYBACK OF RECORDED TRAJECTORIES

gostart(torobo)
setModeOverrides(torobo)


la_min = [70.0, 80.0, -30.0, 0.0]
la_max = [100.0, 90.0,  30.0, 15.0]
DATA_DIR_0='/home/torobo/catkin_ws/src/tutorial/src/data/Joints_9_right_arm/0_1_0_1_0_0_1_1/primitive_0_0.csv'
DATA_DIR_1='/home/torobo/catkin_ws/src/tutorial/src/data/Training_data_2/primitive_1_0.csv'
DATA_DIR_2='/home/torobo/catkin_ws/src/tutorial/src/data/Training_data_3/primitive_2_0.csv'
ra_min = la_min
ra_max = la_max


# Define the minimum and maximum of the head angles
head_min = [-20.0, -10.0]
head_max = [20.0,  20.0]

ARM_JNTS_FIXED = 0.0
TORSO_FIXED = 0.0

# The movement will be generated for 4 seconds
PLAY_DURATION = 10

# We'll capture 10 data points per second
CAPTURE_RATE = 10


print "Demonstrating Recorded Trajectory"
print"PRIMITIVE_0"
cor_saved=np.loadtxt(DATA_DIR_0, delimiter=",")
cor_saved_1=np.loadtxt(DATA_DIR_1, delimiter=",")
cor_saved_2=np.loadtxt(DATA_DIR_2, delimiter="," )
for i in xrange(cor_saved.shape[0]):
    #left_cor=list(cor_saved[i][:6])

    right_cor=list(array_radians(cor_saved[i]))
    #torobo.move(ToroboOperator.LEFT_ARM, positions=left_cor)
    #time.sleep(1)
    torobo.move(ToroboOperator.RIGHT_ARM, positions=right_cor)
    time.sleep(1)
print 'PRIMITIVE_1'

move_startpos(torobo)

for i in xrange(cor_saved_1.shape[0]):
    #left_cor=list(cor_saved_1[i][:6])
    right_cor=list(array_radians(cor_saved_1[i]))
    #torobo.move(ToroboOperator.LEFT_ARM, positions=left_cor)
    torobo.move(ToroboOperator.RIGHT_ARM, positions=right_cor)
    time.sleep(1)
print "PRIMITIVE_2"

move_startpos(torobo)

for i in xrange(cor_saved_2.shape[0]):
    #left_cor=list(cor_saved_2[i][:6])
    #right_cor=list(cor_saved_2[i][6:12])
   # torobo.move(ToroboOperator.LEFT_ARM, positions=left_cor)

    right_cor=list(array_radians(cor_saved_2[i]))
    torobo.move(ToroboOperator.RIGHT_ARM, positions=right_cor)
    time.sleep(1)


In [2]:
# GENERATION OF PRIOR

# prop_path = "/home/torobo/catkin_ws/src/tutorial/src/data/config/model_2_seq_2.d"
prop_path = "/home/torobo/catkin_ws/src/tutorial/src/data/config/modxp.d"
model = Trial(prop_path)

gostart(torobo)
setModeOverrides(torobo)

model_list = []
model_data = np.array(model.generate_pos())

while True:
    try:
        full_cor = np.array(model.generate_pos())
        left_cor = np.radians(full_cor[0:6])
        right_cor = np.radians(full_cor[6:12])

        torobo.move(ToroboOperator.LEFT_ARM, positions=left_cor)
        torobo.move(ToroboOperator.RIGHT_ARM, positions=right_cor)
        time.sleep(0.2)
    except:
        print(colored('user interrupt', 'red', 'on_yellow', attrs=['bold']))
        break
# move_startpos(torobo)

SUCCESS
SETTING MODE: set_external_force_following_online_trajectory_control
SETTING DYNAMIC PARAMETERS
OVERRIDES SET
user interrupt
